In [2]:
import pandas as pd
import sqlite3

In [3]:
conn = sqlite3.connect('bios-players-states.db')

df_bios = pd.read_excel('../processed-data/bios.xlsx', sheet_name='bios')
df_bios.to_sql('bios', conn, index=False,if_exists="replace")

df_players = pd.read_excel('../processed-data/stacked-player-stats.xlsx', sheet_name='player-stats')
df_players.to_sql('players', conn, index=False,if_exists="replace")

df_birth = pd.read_excel('../processed-data/player-birth-stats.xlsx', sheet_name='birth-stats')
df_birth.to_sql('birth', conn, index=False,if_exists="replace")

curr = conn.cursor()

C:\Users\Max Brenner\Anaconda3\lib\site-packages\pandas\core\generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [4]:
df_bios.head()

,Player,Age,Height,Weight,College,Country,Draft Year,Draft Round,Draft Number
0,Aaron Gordon,24,80,235,Arizona,USA,2014.0,1.0,4.0
1,Aaron Holiday,23,72,185,UCLA,USA,2018.0,1.0,23.0
2,Abdel Nader,26,77,225,Iowa State,Egypt,2016.0,2.0,58.0
3,Adam Mokoka,21,77,190,NaN,France,NaN,NaN,NaN
4,Admiral Schofield,23,77,241,Tennessee,United Kingdo ...,2019.0,2.0,42.0


In [5]:
df_players.head()

,Player,Season,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,Steven Adams,19-20,C,26,OKC,49,1322,20.1,0.597,0.008,...,14.6,17.1,3.0,2.0,5.0,0.182,1.0,3.3,4.3,2.1
1,Bam Adebayo,19-20,PF,22,MIA,54,1865,20.7,0.613,0.019,...,17.2,19.9,4.1,3.0,7.1,0.183,1.1,4.0,5.1,3.3
2,LaMarcus Aldridge,19-20,C,34,SAS,50,1661,20.3,0.580,0.195,...,7.9,23.3,3.3,1.3,4.6,0.133,1.4,0.4,1.8,1.6
3,Nickeil Alexander-Walker,19-20,SG,21,NOP,41,501,7.7,0.441,0.515,...,14.3,22.9,-0.7,0.3,-0.4,-0.038,-4.0,-2.1,-6.1,-0.5
4,Grayson Allen,19-20,SG,24,MEM,30,498,11.4,0.577,0.517,...,10.7,18.0,0.5,0.2,0.7,0.065,-0.8,-2.6,-3.4,-0.2


In [6]:
df_birth.head()

,Player,Date,City,State
0,Michael Ansley,"Feb 8, 1967",Birmingham,Alabama
1,Keith Askins,"Dec 15, 1967",Athens,Alabama
2,Carl Bailey,"Apr 23, 1958",Birmingham,Alabama
3,Charles Barkley,"Feb 20, 1963",Leeds,Alabama
4,Eric Bledsoe,1989-12-09 00:00:00,Birmingham,Alabama


In [7]:
def query_db(query):
    execute = curr.execute(query)
    cols = [column[0] for column in execute.description]
    results = pd.DataFrame.from_records(data=execute.fetchall(), columns=cols)
    return results

In [8]:
# Average VORP per state
query = \
'''
SELECT State, AVG(VORP)
FROM players
JOIN birth ON players.player is birth.player
WHERE Season is "19-20"
GROUP BY State
HAVING COUNT(State) >= 5
ORDER BY AVG(VORP) DESC
LIMIT 5
'''
query_db(query)

,State,AVG(VORP)
0,North Carolina,1.018182
1,California,0.986207
2,New York,0.900000
3,Texas,0.800000
4,Ohio,0.742857


In [10]:
# Average Height (inches) per state
query = \
'''
SELECT State, AVG(Height)
FROM bios
JOIN birth ON bios.player is birth.player
GROUP BY State
HAVING COUNT(State) >= 5
ORDER BY AVG(Height) DESC
LIMIT 5
'''
query_db(query)

,State,AVG(Height)
0,New York,79.333333
1,California,78.814815
2,Texas,78.777778
3,Virginia,78.666667
4,Georgia,78.444444
